In [6]:
!pip install fasttext

     |████████████████████████████████| 68 kB 968 kB/s eta 0:00:01
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp39-cp39-macosx_10_16_x86_64.whl size=305024 sha256=3970d8f97611b165570bd0de216c80e6efcfaa9ae1ac7f5dc53ba485e96439b0
  Stored in directory: /Users/przivic/Library/Caches/pip/wheels/64/57/bc/1741406019061d5664914b070bd3e71f6244648732bc96109e
Successfully built fasttext


In [9]:
from taller_model_selection.serialize import iter_jl
X_test = list(iter_jl('X_test.jl'))

In [11]:
test_ids = set(d['id'] for d in X_test)

In [12]:
import csv

with open('ar_properties.csv') as f:
    it = csv.DictReader(f)
    titles = [doc['title'] for doc in it if doc['id'] not in test_ids]

In [ ]:
# maybe some preprocessing?

In [16]:
with open('data.txt', 'w') as f:
    f.writelines(titles)

In [17]:
import fasttext
model = fasttext.train_unsupervised('data.txt', model='skipgram')


Read 6M words
Number of words:  48060
Number of labels: 0
Progress: 100.0% words/sec/thread:   74778 lr:  0.000000 avg.loss:  1.619877 ETA:   0h 0m 0s 22.2% words/sec/thread:   76928 lr:  0.038911 avg.loss:  1.747737 ETA:   0h 0m48s 47.8% words/sec/thread:   78908 lr:  0.026084 avg.loss:  1.678311 ETA:   0h 0m31s 75.8% words/sec/thread:   73968 lr:  0.012120 avg.loss:  1.648272 ETA:   0h 0m15s  73935 lr:  0.007201 avg.loss:  1.634734 ETA:   0h 0m 9s


In [18]:
model.save_model("title_fasttext.bin")